# 9-2 ChatHuggingFace 양자화

In [ ]:
# 필요한 라이브러리 import
from transformers import BitsAndBytesConfig  # 양자화 설정을 위한 transformers 모듈
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline  # HuggingFace 파이프라인 및 채팅 래퍼
import torch  # 디바이스(CPU/GPU) 확인용

# 4비트 양자화 설정
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

# 디바이스 설정: GPU가 있으면 0, 없으면 -1(CPU)
device = 0 if torch.cuda.is_available() else -1

# 허깅페이스 허브에서 모델을 파이프라인 형태로 불러오기
# task를 'text-generation'으로 지정해서 텍스트 생성용 파이프라인을 만든다.
quantized_llm = HuggingFacePipeline.from_model_id(
    model_id="LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct",
    task="text-generation",
    pipeline_kwargs=dict(
      max_new_tokens=1024,
      do_sample=False,
      repetition_penalty=1.03,
    ),
    device=device,  # 0 for first GPU, -1 for CPU
    model_kwargs={"quantization_config": quantization_config},
)

# 생성된 파이프라인을 ChatHuggingFace에 전달해서 채팅 형태로 사용할 수 있다.
quantized_llm = ChatHuggingFace(llm=quantized_llm)

In [ ]:
quantized_ai_message = quantized_llm.invoke("RAG(Retrieval Augmented Generation) 파이프라인은 무엇인가요?")

In [1]:
quantized_ai_message

AIMessage(content='RAG(Retrieval Augmented Generation) 파이프라인은 최근 자연어 처리(NLP) 분야에서 주목받고 있는 모델 아키텍처입니다. 이 접근 방식은 기존의 언어 생성 모델을 개선하여 더 정확하고 관련성 높은 응답을 생성하기 위해 정보 검색과 결합된 구조를 가지고 있습니다.\n\nRAG 파이프라인의 주요 구성 요소는 다음과 같습니다:\n\n1. 정보 검색 모듈 (Retrieval Module):\n   - 입력 질문이나 프롬프트에 대해 데이터베이스나 문서 집합에서 관련 정보를 검색합니다.\n   - 검색된 정보는 텍스트 형태로 반환되며, 이를 기반으로 후속 작업이 진행됩니다.\n\n2. 언어 모델 (Language Model):\n   - 검색된 정보를 바탕으로 텍스트를 생성하거나 요약합니다.\n   - 기존의 트랜스포머 기반 언어 모델(예: GPT-3, BERT 등)을 사용할 수 있으며, 검색된 정보와 함께 학습 및 추론 과정을 거칩니다.\n\n3. 통합 및 응답 생성 모듈:\n   - 정보 검색 결과와 언어 모델의 출력을 통합하여 최종 응답을 생성합니다.\n   - 이 단계에서는 두 가지 정보 소스를 효과적으로 결합해 보다 풍부하고 정확한 답변을 제공합니다.\n\nRAG 파이프라인의 장점:\n- 정확성 향상: 필요한 정보를 먼저 찾아서 활용함으로써 답변의 정확성을 높일 수 있습니다.\n- 효율성 증대: 특정 주제에 대한 깊이 있는 지식을 빠르게 제공할 수 있습니다.\n- 다양한 응용 가능성: 문서 요약, 질의응답 시스템, 지식 기반 챗봇 등 다양한 NLP 응용 분야에 적용할 수 있습니다.\n\nRAG는 특히 대규모 데이터셋에서 유용하며, 사용자가 원하는 정보를 빠르게 찾고 이를 바탕으로 고품질의 응답을 생성하는 데 강점을 보입니다.')